In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
inflows = pd.read_parquet("../data/ucsd-inflows.pqt")
outflows = pd.read_parquet("../data/ucsd-outflows.pqt")

In [18]:
# start with outflows

for column in outflows.columns:
    current_column = outflows[column]
    dtypes, nulls, shape, unique_values = current_column.dtypes, np.sum(current_column.isnull()), current_column.shape, len(current_column.unique())
    column_description = current_column.describe()
    print(f"Column: {column}: \nDtypes of {dtypes}")
    print(f"Null Values: {nulls}")
    print(f"Shape: {shape}")
    print(f"Unique Values: {unique_values}")
    print(f"Description: {column_description}\n")



Column: prism_consumer_id: 
Dtypes of int64
Null Values: 0
Shape: (2597488,)
Unique Values: 2968
Description: count    2.597488e+06
mean     3.084229e+03
std      1.820571e+03
min      0.000000e+00
25%      1.369000e+03
50%      3.245000e+03
75%      4.733000e+03
max      5.943000e+03
Name: prism_consumer_id, dtype: float64

Column: prism_account_id: 
Dtypes of object
Null Values: 0
Shape: (2597488,)
Unique Values: 4622
Description: count      2597488
unique        4622
top       acc_9475
freq          5470
Name: prism_account_id, dtype: object

Column: memo: 
Dtypes of object
Null Values: 0
Shape: (2597488,)
Unique Values: 528786
Description: count               2597488
unique               528786
top       EXTERNAL_TRANSFER
freq                 320998
Name: memo, dtype: object

Column: amount: 
Dtypes of float64
Null Values: 0
Shape: (2597488,)
Unique Values: 72262
Description: count    2.597488e+06
mean     1.451264e+02
std      1.697875e+03
min      0.000000e+00
25%      9.630000e+

In [ ]:
for column in out